In [2]:
! pip install kaggle

In [3]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [4]:
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 98% 79.0M/80.9M [00:04<00:00, 24.1MB/s]
100% 80.9M/80.9M [00:04<00:00, 18.5MB/s]


In [5]:
from zipfile import ZipFile
dataset = "/content/sentiment140.zip"
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("Dataset is extracted")

Dataset is extracted


In [6]:
import numpy as np
import pandas as pd
import re      # Regular expression used for doing some pattern matching
from nltk.corpus import stopwords     #natural language toolkit
from nltk.stem.porter import PorterStemmer                 #Steming we do to return a word to its root word
from sklearn.feature_extraction.text import TfidfVectorizer          #Vectorizes the textual data so we can feed this to the modle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
print(stopwords.words('english'))       # Dosent have any influential meaning to the data

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
column_names = ['Target','Id','Date','Flag','User','Text']
twitter_data =pd.read_csv('/content/training.1600000.processed.noemoticon.csv' ,names = column_names, encoding = 'ISO-8859-1')

In [10]:
twitter_data.head()


,Target,Id,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [11]:
# Data handeling and feature engineering
twitter_data.isnull().sum()

,0
Target,0
Id,0
Date,0
Flag,0
User,0
Text,0


In [12]:
twitter_data['Target'].value_counts()

,count
Target,
0,800000
4,800000


In [13]:
twitter_data.replace({'Target':{4:1}},inplace=True)     #Inplace = true makes so the changes happen in the original database only , no new database is created

In [14]:
port_stem = PorterStemmer()

def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content


In [ ]:
twitter_data['stemmed_content'] = twitter_data['Text'].apply(stemming)
# 50 min for this

In [ ]:
twitter_data.head()

In [ ]:
twitter_data['stemmed_content']
twitter_data['Target']

In [ ]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['Target'].values

In [ ]:
X_train , X_test , Y_train , Y_test = train_test_split(X , Y , stratify =Y , random_state = 2)
# The proportion of 0 and 1 is same in both test and train ->stratify
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)


In [ ]:
# Convertin textual data to numerical data
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)          # test data is transformed according to the train data so fit_transform should not be used

In [ ]:
#Training the machine learning model

model = LogisticRegression(max_iter = 1000)
model.fit(X_train,Y_train)

#

In [ ]:
# Model evaluation

X_train_pred = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_pred,Y_train)
print(training_data_accuracy)

In [ ]:
X_test_pred = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_pred,Y_test)
print(testing_data_accuracy)

In [ ]:
# Saving the model

import pickle
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))